In [1]:
import pandas as pd
data = pd.read_parquet('/drive2/phatnt/zTrans/data/no_comment_dataset.parquet')

In [ ]:
def remove_comments(source):
    source = source.replace('\r\n', '\n')
    state = "ETC"
    i = 0
    comments = []
    current_comment = None

    while i + 1 < len(source):
        if state == "ETC" and source[i] == '/' and source[i + 1] == '/':
            state = "LINE_COMMENT"
            current_comment = {
                "type": "LineComment",
                "range": {"start": i}
            }
            i += 2
            continue

        if state == "LINE_COMMENT" and source[i] == '\n':
            state = "ETC"
            current_comment["range"]["end"] = i
            comments.append(current_comment)
            current_comment = None
            i += 1
            continue

        if state == "ETC" and source[i] == '/' and source[i + 1] == '*':
            state = "BLOCK_COMMENT"
            current_comment = {
                "type": "BlockComment",
                "range": {"start": i}
            }
            i += 2
            continue

        if state == "BLOCK_COMMENT" and source[i] == '*' and source[i + 1] == '/':
            state = "ETC"
            current_comment["range"]["end"] = i + 2
            comments.append(current_comment)
            current_comment = None
            i += 2
            continue

        i += 1

    # Handle unfinished line comment at the end of the source code
    if current_comment and current_comment["type"] == "LineComment":
        if source[-1] == '\n':
            current_comment["range"]["end"] = len(source) - 1
        else:
            current_comment["range"]["end"] = len(source)
        comments.append(current_comment)

    # Remove the comments from the source code
    def remove_content(source, comments):
        result = []
        last_index = 0

        for comment in comments:
            start = comment["range"]["start"]
            end = comment["range"]["end"]
            # Append code before the comment
            result.append(source[last_index:start])
            # Update last index to skip the comment
            last_index = end

        # Append the remaining source code after the last comment
        result.append(source[last_index:])
        # Join and split lines, then filter out blank lines
        cleaned_code = ''.join(result).split('\n')
        return '\n'.join(line for line in cleaned_code if line.strip())

    return remove_content(source, comments)

In [ ]:
import javalang
def extract_methods_with_body(java_code):
    # print('java_code :',java_code)
    try:
        # Phân tích cú pháp mã Java
        tree = javalang.parse.parse(java_code)
        methods = []
        lines = java_code.splitlines()

        # Tìm các MethodDeclaration
        for _, node in tree.filter(javalang.tree.MethodDeclaration):
            # Lấy dòng bắt đầu của phương thức
            start_line = node.position.line - 1  # 0-based index

            # Xây dựng khai báo hàm đầy đủ (kể cả đa dòng)
            open_parens = 0
            method_declaration = []
            for line in lines[start_line:]:
                method_declaration.append(line)
                open_parens += line.count("(")
                open_parens -= line.count(")")
                if open_parens == 0 and ')' in line:  # Kết thúc khai báo hàm
                    break

            # Chuyển khai báo hàm thành chuỗi
            method_declaration_text = "\n".join(method_declaration).strip()

            # Lấy phần thân hàm (kể cả đa dòng)
            method_body = []
            open_braces = 0
            for line in lines[start_line:]:
                # print('line :',line)
                method_body.append(line)
                open_braces += line.count("{")
                open_braces -= line.count("}")
                if open_braces == 0 and len(method_body) > 1 and '}' in line:
                    break

            # Ghép lại toàn bộ hàm
            full_method_text =  "\n".join(method_body).strip()
            methods.append(full_method_text)

        return methods
    except Exception as e:
        print(e)
        return None
# methods = extract_methods_with_body(java_code)

# # Hiển thị kết quả
# for method in methods:
#     print(method)
#     print("-" * 40)

In [3]:
def diff_methods(methods_start, methods_end):
    # Chuẩn hóa các phương thức để loại bỏ khoảng trắng không cần thiết
    normalized_start = [method.strip() for method in methods_start]
    normalized_end = [method.strip() for method in methods_end]

    # Tạo tập hợp để so sánh
    set_start = set(normalized_start)
    set_end = set(normalized_end)

    # Phương thức bị xóa hoặc thay đổi
    removed_methods = set_start - set_end

    # Phương thức mới hoặc thay đổi
    added_methods = set_end - set_start

    # Phương thức không thay đổi
    unchanged_methods = set_start & set_end

    return {
        "removed": removed_methods,
        "added": added_methods,
        "unchanged": unchanged_methods,
    }
# methods_start = extract_methods_with_body(data.iloc[1]['startCode'])
# methods_end = extract_methods_with_body(data.iloc[1]['endCode'])
def check_same_methods(method1, method2):
    name_method1 = method1.split('(')[0]
    name_method2 = method2.split('(')[0]
    return name_method1 == name_method2

# diff_result = diff_methods(methods_start, methods_end)
# row_changes = []
# removed = diff_result["removed"]
# added = diff_result["added"]

# remaining_added = list(added)

# for method1 in removed:
#     for method2 in list(remaining_added):  # Duyệt qua một bản sao của remaining_added
#         if check_same_methods(method1, method2):
#             row_changes.append([method1, method2])
#             remaining_added.remove(method2)  # Xóa method2 khỏi remaining_added
# for containmethods in remaining_added:
#     row_changes.append(['',containmethods])
# for i in row_changes:
#     for ex in i:
#         print(ex)
#         print('****')
#     print('----->')

# # Hiển thị kết quả
# print("Removed Methods:")
# for method in diff_result["removed"]:
#     print(method)
# print("-" * 40)

# print("Added Methods:")
# for method in diff_result["added"]:
#     print(method)
# print("-" * 40)


In [ ]:
import pandas as pd

def process_and_expand_data(data):
    """
    Hàm xử lý và mở rộng DataFrame gốc bằng cách thêm các cột methods_before và methods_after.
    """
    expanded_data = []
    count = 0

    for index, row in data.iterrows():
        try:

            startCode = row['startCode']
            endCode = row['endCode']


            methods_start = extract_methods_with_body(startCode)
            methods_end = extract_methods_with_body(endCode)


            diff_result = diff_methods(methods_start, methods_end)
            removed = diff_result["removed"]
            added = diff_result["added"]

            row_changes = []
            remaining_added = list(added)

            for method1 in removed:
                have_func = False
                for method2 in list(remaining_added):
                    if check_same_methods(method1, method2):
                        row_changes.append([method1, method2])
                        remaining_added.remove(method2)
                        have_func = True
                        break
                if have_func == False:
                    row_changes.append([method1,''])

            for containmethods in remaining_added:
                row_changes.append(['',containmethods])


            for method1, method2 in row_changes:
                expanded_data.append({
                    **row.to_dict(),
                    'methods_before': method1,
                    'methods_after': method2
                })

        # for method1 in removed:
        #     if not any(method1 == change[0] for change in row_changes):
        #         expanded_data.append({
        #             **row.to_dict(),
        #             'methods_before': method1,
        #             'methods_after': ''
        #         })

        # for method2 in remaining_added:
        #     expanded_data.append({
        #         **row.to_dict(),
        #         'methods_before': '',
        #         'methods_after': method2
        #     })
        except Exception as e:
            # print(e)
            # import traceback
            # traceback.print_exc()
            print('Err at index:',count)
            print('--->')
        count+=1

    expanded_df = pd.DataFrame(expanded_data)
    return expanded_df


expanded_data = process_and_expand_data(data)

print(expanded_data)

'NoneType' object is not iterable
Err at index: 8
--->
'NoneType' object is not iterable
Err at index: 34
--->
'NoneType' object is not iterable
Err at index: 88
--->
'NoneType' object is not iterable
Err at index: 89
--->


: 

: 

In [ ]:
expanded_data['methods_before'] = expanded_data['methods_before'].apply(lambda x: remove_comments(x))
expanded_data['methods_after'] = expanded_data['methods_after'].apply(lambda x: remove_comments(x))

In [8]:
expanded_data.to_parquet('/drive2/phatnt/zTrans/data/data_method.parquet')

In [1]:
import pandas as pd
data  = pd.read_parquet('/drive2/phatnt/zTrans/data/data_method_8k.parquet')
data

,id,fromLib,toLib,repoName,repoOwner,repoSplitName,prevCommit,startCommit,endCommit,fileName,...,lib_percentage,annotation_change,method_changes,method_signature_changes,count_method_changes,count_method_signature_changes,import_before,import_after,methods_before,methods_after
0,3,ant:ant,org.apache.ant:ant,bobmcwhirter_drools,bobmcwhirter,drools,2ba5d35fb486c4a16b5b8b15fc247e8759bfa54f,92384035a8651b675c82689b24837eb8adb81d66,0df25aead759b3b542a685ac21a3a009bcd22fe9,drools-compiler/src/main/java/org/drools/rule/...,...,0.000000,0.0,private final Pattern lineBreakFinder = Patter...,[ private void rewriteModify(final RuleBu...,1,1,import java.util.Arrays;\nimport java.util.Col...,import java.util.Arrays;\nimport java.util.Col...,private void rewriteModify(final RuleBuildCont...,private void rewriteModify(final RuleBuildCont...
1,4,ant:ant,org.apache.ant:ant,bobmcwhirter_drools,bobmcwhirter,drools,2ba5d35fb486c4a16b5b8b15fc247e8759bfa54f,92384035a8651b675c82689b24837eb8adb81d66,0df25aead759b3b542a685ac21a3a009bcd22fe9,drools-compiler/src/test/java/org/drools/guvno...,...,0.096774,0.0,protected void setUp() throws Exception {\nsup...,[ public void testAddSentenceMultipleType...,1,1,import junit.framework.TestCase;\nimport org.d...,import junit.framework.TestCase;\nimport org.d...,public void testAddSentenceMultipleTypes() {\n...,public void testAddSentenceMultipleTypes() {\n...
2,7,ant:ant,org.apache.ant:ant,bobmcwhirter_drools,bobmcwhirter,drools,2ba5d35fb486c4a16b5b8b15fc247e8759bfa54f,92384035a8651b675c82689b24837eb8adb81d66,0df25aead759b3b542a685ac21a3a009bcd22fe9,drools-compiler/src/test/java/org/drools/integ...,...,0.000000,0.0,protected RuleBase getRuleBase() throws Except...,[ public void testLockOnActiveWithModify2...,2,2,import java.io.IOException;\nimport java.io.In...,import java.io.IOException;\nimport java.io.In...,public void testUpdateActivationCreationNoLoop...,public void testUpdateActivationCreationNoLoop...
3,7,ant:ant,org.apache.ant:ant,bobmcwhirter_drools,bobmcwhirter,drools,2ba5d35fb486c4a16b5b8b15fc247e8759bfa54f,92384035a8651b675c82689b24837eb8adb81d66,0df25aead759b3b542a685ac21a3a009bcd22fe9,drools-compiler/src/test/java/org/drools/integ...,...,0.000000,0.0,protected RuleBase getRuleBase() throws Except...,[ public void testLockOnActiveWithModify2...,2,2,import java.io.IOException;\nimport java.io.In...,import java.io.IOException;\nimport java.io.In...,public void testLockOnActiveWithModify2() thro...,public void testLockOnActiveWithModify2() thro...
4,9,ant:ant,org.apache.ant:ant,bobmcwhirter_drools,bobmcwhirter,drools,2ba5d35fb486c4a16b5b8b15fc247e8759bfa54f,92384035a8651b675c82689b24837eb8adb81d66,0df25aead759b3b542a685ac21a3a009bcd22fe9,drools-compiler/src/test/java/org/drools/integ...,...,0.227273,0.0,protected RuleBase getRuleBase() throws Except...,[ public void testAddRemoveListeners() th...,6,6,import java.io.IOException;\nimport java.io.In...,import static org.mockito.Matchers.any;\nimpor...,public void testAssertRetract() throws Excepti...,public void testAssertRetract() throws Excepti...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31789,33973,com.google.inject:guice,com.google.dagger:dagger,spotify_heroic,spotify,heroic,a0898aba0e9c16ce6c6b5e90df0a0f0344b14888,c342ebe32a8c6b7daa12f053a8bfac8b8ce06760,c342ebe32a8c6b7daa12f053a8bfac8b8ce06760,heroic-core/src/test/java/com/spotify/heroic/H...,...,0.000000,0.0,private final ObjectMapper mapper = HeroicMapp...,[ public InputStream resource(final Strin...,1,1,import com.fasterxml.jackson.databind.ObjectMa...,import com.fasterxml.jackson.databind.ObjectMa...,public InputStream resource(final String name)...,public InputStream resource(final String name)...
31790,33974,com.google.inject:guice,com.google.dagger:dagger,spotify_heroic,spotify,heroic,a0898aba0e9c16ce6c6b5e90df0a0f0344b14888,c342ebe32a8c6b7daa12f053a8bfac8b8ce06760,c342ebe32a8c6b7daa12f053a8bfac8b8ce06760,heroic-core/src/test/java/com/spotify/he

In [3]:
data= data[data['methods_before'] == '']
data

,id,fromLib,toLib,repoName,repoOwner,repoSplitName,prevCommit,startCommit,endCommit,fileName,...,lib_percentage,annotation_change,method_changes,method_signature_changes,count_method_changes,count_method_signature_changes,import_before,import_after,methods_before,methods_after
13,12,ant:ant,org.apache.ant:ant,bobmcwhirter_drools,bobmcwhirter,drools,2ba5d35fb486c4a16b5b8b15fc247e8759bfa54f,92384035a8651b675c82689b24837eb8adb81d66,0df25aead759b3b542a685ac21a3a009bcd22fe9,drools-compiler/src/test/java/org/drools/reteo...,...,0.100000,0.000000,private final List asserted = n...,[+ public void modifyLeftTuple(InternalFac...,1,2,import java.util.ArrayList;\nimport java.util....,import java.util.ArrayList;\nimport java.util....,,public void modifyLeftTuple(LeftTuple leftTupl...
14,12,ant:ant,org.apache.ant:ant,bobmcwhirter_drools,bobmcwhirter,drools,2ba5d35fb486c4a16b5b8b15fc247e8759bfa54f,92384035a8651b675c82689b24837eb8adb81d66,0df25aead759b3b542a685ac21a3a009bcd22fe9,drools-compiler/src/test/java/org/drools/reteo...,...,0.100000,0.000000,private final List asserted = n...,[+ public void modifyLeftTuple(InternalFac...,1,2,import java.util.ArrayList;\nimport java.util....,import java.util.ArrayList;\nimport java.util....,,public void modifyLeftTuple(InternalFactHandle...
15,13,ant:ant,org.apache.ant:ant,bobmcwhirter_drools,bobmcwhirter,drools,2ba5d35fb486c4a16b5b8b15fc247e8759bfa54f,92384035a8651b675c82689b24837eb8adb81d66,0df25aead759b3b542a685ac21a3a009bcd22fe9,drools-compiler/src/test/java/org/drools/reteo...,...,0.000000,0.000000,private final List retracted = n...,[+ public void modifyRightTuple(RightTuple...,2,2,import java.io.IOException;\nimport java.io.Ob...,import java.io.IOException;\nimport java.io.Ob...,,public void modifyRightTuple(RightTuple rightT...
27,21,ant:ant,org.apache.ant:ant,bobmcwhirter_drools,bobmcwhirter,drools,2ba5d35fb486c4a16b5b8b15fc247e8759bfa54f,92384035a8651b675c82689b24837eb8adb81d66,0df25aead759b3b542a685ac21a3a009bcd22fe9,drools-core/src/main/java/org/drools/base/Clas...,...,0.333333,0.000000,private Set<String> imports = Co...,"[+ public void clearImports() {, ...",2,2,import java.util.Collections;\nimport java.uti...,import java.util.Collections;\nimport java.uti...,,public void clearImports() {\n if( this...
40,25,ant:ant,org.apache.ant:ant,bobmcwhirter_drools,bobmcwhirter,drools,2ba5d35fb486c4a16b5b8b15fc247e8759bfa54f,92384035a8651b675c82689b24837eb8adb81d66,0df25aead759b3b542a685ac21a3a009bcd22fe9,drools-core/src/main/java/org/drools/common/Ag...,...,0.000000,0.000000,public AgendaItem() {\n\n}\n\npublic AgendaIte...,"[ public int getSalience() {, + publi...",4,4,import java.io.Externalizable;\nimport java.io...,import java.io.Externalizable;\nimport java.io...,,public void setSalience(int salience) {\n ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31780,33967,com.google.inject:guice,com.google.dagger:dagger,spotify_heroic,spotify,heroic,a0898aba0e9c16ce6c6b5e90df0a0f0344b14888,c342ebe32a8c6b7daa12f053a8bfac8b8ce06760,c342ebe32a8c6b7daa12f053a8bfac8b8ce06760,heroic-core/src/main/java/com/spotify/heroic/w...,...,0.101695,0.101695,,[ * Licensed to the Apache Software Foundati...,3,6,import com.google.inject.Inject;\nimport com.s...,import com.google.common.collect.ImmutableList...,,WebApplicationExceptionMapper webApplicationEx...
31781,33967,com.google.inject:guice,com.google.dagger:dagger,spotify_heroic,spotify,heroic,a0898aba0e9c16ce6c6b5e90df0a0f0344b14888,c342ebe32a8c6b7daa12f053a8bfac8b8ce06760,c342ebe32a8c6b7daa12f053a8bfac8b8ce06760,heroic-core/src/main/java/com/spotify/heroic/w...,...,0.101695,0.101695,,[ * Licensed to the Apache Software Foundati...,3,6,import com.google.inject.Inject;\nimport com.s...,import com.google.common.collect.ImmutableList...,,ParseExceptionMapper parseExceptionMapper();\n...
31782,33967,com.google.inject:guice,com.google.dagger:dagger,spotify_heroic,spotify,heroic,a0898aba0e9c16ce6c6b5e90df0a0f0344b14888,c342eb